In [1]:
import shutil
try:
  shutil.copy('/content/dataset.zip', '/content/')
  print('dataset download!!!')
except Exception as err:
  print(str(err))

'/content/dataset.zip' and '/content/dataset.zip' are the same file


In [2]:
import zipfile

with zipfile.ZipFile('/content/dataset.zip', 'r') as target_file:
    target_file.extractall('/content/dataset/')
print('ok')

ok


In [3]:
!pip install ujson
!pip install gensim
!pip install konlpy
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.7 MB/s 
     |████████████████████████████████| 453 kB 58.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=d082812233d40b11de04e0ca5702245f56d883891191104d7ab8a227e4ce53cc
  Stored in directory: /root/.cache/pip/wheels/

In [4]:
# Data Preprocessing Package
import re                       #정규식
import numpy as np
import pandas as pd
import os                      #디렉토리와 경로정보

# NLP Package
from konlpy.tag import * 
import gensim                    #토픽모델링을 하는 라이브러리
import gensim.corpora as corpora #텍스트분석
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package   #LDA시각화
import pyLDAvis 
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint #pretty print
import itertools #iterable 객체 처리
import math

import logging #로그처리
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings #경고 무시
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [5]:
ROOT_PATH ="./dataset/"

DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"1.Data_my/")
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"text.xlsx")                  
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"Data_stop_word.txt")  

MODEL_SAVE_FOLDER = os.path.join(ROOT_PATH,"2.Model_my/")
LDA_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_lda_topic_modeling.lda")
OPT_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_opt_topic_modeling.lda")

RESULT_FOLDER = os.path.join(ROOT_PATH,"3.Result_my/")
RESULT_SAVE_LDAVIS = os.path.join(RESULT_FOLDER,"Result_lda_vis.html")
RESULT_TOPIC_EXCEL = os.path.join(RESULT_FOLDER,"Result_topic_excel.xlsx")
RESULT_corpus_EXCEL = os.path.join(RESULT_FOLDER,"Result_corpus_excel.xlsx")

In [6]:
DF_raw = pd.read_excel(DATA_FILE_NAME,sheet_name='Sheet1') 
len(DF_raw)

print('Null값이 있는지 확인합니다.',DF_raw.isnull().values.any()) 
print('')
DF_raw = DF_raw.dropna(how = 'any') 
DF_raw = DF_raw.drop_duplicates()    
DF_raw = DF_raw.reset_index(drop=True) 
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', DF_raw.isnull().values.any()) 
print('')
print("처리할 데이터수 : ",len(DF_raw))

DF_only_text = DF_raw['text'] 

Null값이 있는지 확인합니다. False

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False

처리할 데이터수 :  35


In [7]:
tokenizer = Okt() 
stopword_vocab = DATA_STOP_WORD_FILE_NAME 
sep = "\n" 

def build_vocab(data_frame ,stopword_vocab, separate):
    
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
     
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data =build_vocab(DF_only_text, stopword_vocab, sep)



def word_corpus(result_data):
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) 
    return vocab

vocab=word_corpus(result_data)

df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1

a=df_corpus.head(1000)
print(df_corpus.head(20))

a.to_excel(RESULT_corpus_EXCEL ,sheet_name = "sheet1")

id2word = corpora.Dictionary(result_data)
 
texts = result_data
 
corpus = [id2word.doc2bow(text) for text in texts]

전체 워드의 개수 : 6401
     text  count
0      국민     62
1      대한     45
2      투표     40
3      개정     38
4     노동자     37
5      정책     37
6      국가     37
7      인권     37
8      여성     36
9      가족     35
10  국가인권위     34
11     이유     33
12     사회     33
13     교육     30
14     참사     30
15     모든     29
16    공무원     29
17     폐지     27
18     조사     27
19     위해     27


In [8]:
df_corpus # <- 나중에 그래프로 시각화 

,text,count
0,국민,62
1,대한,45
2,투표,40
3,개정,38
4,노동자,37
...,...,...
1945,신앙,1
1946,학문,1
1947,강행,1
1948,세대,1


In [9]:
TOPICS_W_NUM =20
save_lda_model=0
RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,51):
  NUM_TOPICS=i
 
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

  doc_lda = lda_model[corpus]

  if save_lda_model == 0:
      lda_model.save(LDA_MODEL_SAVE_NAME)

  #print('\nNUM_TOPICS',NUM_TOPICS)
  # Perplexity 
  #print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
  
  # Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  #print('Coherence Score: ', coherence_lda)
  #print('NUM_TOPICS',NUM_TOPICS,'Perplexity: ', lda_model.log_perplexity(corpus),'Coherence: ', coherence_lda)

  print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)

NUM_TOPICS perplexity coherence
T 1 -7.279923543094229 0.40598060487241305
T 2 -7.2024058974290535 0.37078062522285493
T 3 -7.15051904922199 0.4321807899297753
T 4 -7.123008500529144 0.37912067842518304
T 5 -7.127974538698529 0.41058233559616975
T 6 -7.085487704882955 0.41962850449552436
T 7 -7.067847964412494 0.4507384044686965
T 8 -7.0592337195490735 0.3862569131977026
T 9 -7.041287014408422 0.4079332257828937
T 10 -7.0201593367611315 0.42620844649546924
T 11 -7.021576222447265 0.4528334621877232
T 12 -7.048063387127336 0.4363906643586067
T 13 -7.016722959375925 0.45794715475324266
T 14 -7.001225219380388 0.5068339118772153
T 15 -7.0216226231454035 0.4080456198650479
T 16 -6.9760409557452485 0.5196144211391651
T 17 -7.039738193049578 0.5042440458984037
T 18 -7.050801780080072 0.4790223706560601
T 19 -7.088528981545515 0.4807975385502885
T 20 -7.013054267621901 0.48892087593560146
T 21 -7.073269416384167 0.4483066299777318
T 22 -7.0933432204055515 0.5020827640958729
T 23 -7.0107940185

In [10]:
print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
print(' ')

NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                            num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                            update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                            passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
doc_lda = lda_model[corpus]

if save_lda_model == 0:
    lda_model.save(LDA_MODEL_SAVE_NAME)

토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.
 
토픽의 개수를 입력해 주세요. 39
출력할 토픽별 단어의 개수를 입력해 주세요 10
선택한 토픽 모델을 저장하시겠습니까? 
0 저장  
1 미저장  0
[(37,
  '0.001*"당선인" + 0.001*"의혹" + 0.001*"윤석열" + 0.001*"국민" + 0.001*"검증" + '
  '0.001*"납득" + 0.001*"신분" + 0.001*"대한" + 0.001*"도저히" + 0.001*"수사"'),
 (0,
  '0.003*"농민" + 0.003*"농업" + 0.002*"식량" + 0.002*"농지" + 0.002*"자급" + 0.002*"한국" '
  '+ 0.002*"투기" + 0.001*"등록" + 0.001*"농업인" + 0.001*"규정"'),
 (20,
  '0.001*"가족" + 0.001*"지원" + 0.001*"의사" + 0.001*"참사" + 0.001*"개정안" + '
  '0.001*"동거" + 0.001*"조사" + 0.001*"건강" + 0.001*"통과" + 0.001*"가정"'),
 (17,
  '0.001*"간호사" + 0.001*"환자" + 0.001*"여성" + 0.001*"대한" + 0.001*"차별" + '
  '0.001*"또한" + 0.001*"법적" + 0.001*"간호" + 0.001*"때문" + 0.001*"비율"'),
 (15,
  '0.001*"평범" + 0.001*"국회" + 0.001*"사회" + 0.001*"이유" + 0.001*"인식" + 0.001*"국민" '
  '+ 0.001*"대한" + 0.001*"차별" + 0.001*"투표" + 0.001*"금지법"'),
 (4,
  '0.001*"교육" + 0.001*"중등교육" + 0.001*"법적" + 0.001*"무직" + 0.001*"개정" + '
  '0.001*"학교" + 0.001*"근거" + 0.001*"방과후" + 0.001*"마련" + 

In [11]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# Perplexity는 작을 수록 Coherence Score는 높을 수록 좋다.모델 1개의 값
# 토픽의 개수를 다르게 하여 판단비교해보세요. 동시에 높아지니 믿어도 되겠다. 하나만 선택해라면 코히어런스를 써
# 젠심 코히러런스로 검색해봐서 coherence='c_v'값을 바꿔가면서 해보세요


Perplexity:  -6.980912229024401

Coherence Score:  0.5449025832457419


In [12]:
for topic_id in range(NUM_TOPICS):
    topic_word_probs = lda_model.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    for topic_word, prob in topic_word_probs:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	농민	0.0026402424555271864
	농업	0.002624619286507368
	식량	0.0023880628868937492
	농지	0.0018514774274080992
	자급	0.0017212082166224718
	한국	0.0016699479892849922
	투기	0.0015616252785548568
	등록	0.001433838508091867
	농업인	0.0013513359008356929
	규정	0.001191516756080091


Topic ID: 1
	노동자	0.0007836215663701296
	기업	0.0007656174129806459
	가족	0.0007508769631385803
	처벌	0.0007171861361712217
	인권	0.0006962772458791733
	재해	0.0006789707113057375
	참사	0.0006734721828252077
	중대	0.0006698389770463109
	국가인권위	0.0006518663722090423
	정책	0.0006445475155487657


Topic ID: 2
	노동자	0.03780905902385712
	노동	0.02225620672106743
	국민	0.02005143091082573
	보장	0.01781396195292473
	적용	0.0178061556071043
	조건	0.015598258934915066
	사업	0.015595601871609688
	사용자	0.015593604184687138
	수사	0.013395686633884907
	모든	0.01337534561753273


Topic ID: 3
	이유	0.0013463187497109175
	폐지	0.0012142849154770374
	투표	0.0010823862394317985
	전력	0.0010392401600256562
	가족	0.0009696243796497583
	민간	0.0009519003797322512
	의사	0.00090066739358007

In [13]:
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    pyLDAvis.save_html(vis, RESULT_SAVE_LDAVIS)
    return vis
#lda_model or optimal_model
create_vis(lda_model)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.039937 -0.032490       1        1  0.008642
1      0.040052 -0.033703       2        1  0.009180
2      0.003821  0.161039       3        1  6.225769
3      0.033650 -0.034649       4        1  0.010042
4      0.040565 -0.037032       5        1  0.009146
5     -0.036045  0.104905       6        1  4.310226
6     -0.062312  0.050712       7        1  5.671497
7      0.072086  0.173923       8        1  4.632885
8      0.035036 -0.037236       9        1  0.009219
9      0.037708 -0.033206      10        1  0.009851
10     0.033546 -0.035638      11        1  0.010036
11     0.018042  0.013431      12        1  2.575572
12    -0.009701  0.008417      13        1  3.349253
13     0.029396 -0.076722      14        1  3.310368
14     0.039396 -0.034158      15        1  0.009311
15     0.040518 -0.035831      16        1  0.008992
16    -0.127252 -0.092344      17        1  4.027068
17     0.038526 -0.036903      18        1  0.008993
18    -0.202800 -0.063885      19        1  8.858011
19    -0.085180  0.061493      20        1  4.471204
20     0.039239 -0.037163      21        1  0.008949
21    -0.209798  0.062662      22        1  5.134429
22     0.081185 -0.044788      23        1  1.545045
23     0.097413 -0.032157      24        1  1.683345
24     0.035394 -0.039390      25        1  0.009307
25     0.022799  0.043497      26        1  3.831687
26     0.046098 -0.073787      27        1  2.078632
27    -0.066949  0.065496      28        1  8.225561
28    -0.120312  0.025851      29        1  7.065077
29    -0.052422  0.114930      30        1  4.580769
30    -0.020003  0.034732      31        1  3.606811
31     0.035476 -0.030839      32        1  0.009182
32     0.116099  0.022935      33        1  1.530250
33     0.010126 -0.060187      34        1  1.935590
34    -0.036078 -0.036570      35        1  2.206934
35     0.093444  0.169270      36        1  4.723302
36     0.039887 -0.033043      37        1  0.009174
37     0.041863 -0.036924      38        1  0.008323
38    -0.132450 -0.104650      39        1  4.282371, topic_info=       Term       Freq      Total Category  logprob  loglift
725      투표  37.000000  37.000000  Default  30.0000  30.0000
405      가족  29.000000  29.000000  Default  29.0000  29.0000
533      인권  32.000000  32.000000  Default  28.0000  28.0000
466   국가인권위  30.000000  30.000000  Default  27.0000  27.0000
1917     의사  19.000000  19.000000  Default  26.0000  26.0000
...     ...        ...        ...      ...      ...      ...
214      포함   3.407342  15.516044  Topic39  -4.3876   1.6347
868      정의   2.550807   9.058338  Topic39  -4.6771   1.8834
116      국가   2.552591  31.966658  Topic39  -4.6764   0.6231
773      인정   2.551149   9.935173  Topic39  -4.6770   1.7911
528      의원   2.550116  10.135344  Topic39  -4.6774   1.7708

[2605 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1693     29  0.868428   가격
167      17  0.408151  가능성
167      22  0.272100  가능성
167      28  0.136050  가능성
167      30  0.136050  가능성
...     ...       ...  ...
110      12  0.784335  후보자
1810      8  0.735516   후속
1498      6  0.739973   훼손
816       3  0.717500   휴가
817       3  0.717368   휴게

[2417 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])

In [14]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        topics_info_by_doc = row[0]
        topics_info_by_doc = sorted(topics_info_by_doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(topics_info_by_doc):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]),
                                                       ignore_index=True)
            else:
                break

    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data)
topic_weight=lda_model[corpus]
df_topic_weight = pd.DataFrame()
for i in range(1, lda_model.num_topics+1):
    df_topic_weight['topic{}'.format(i)]=pd.Series()


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 

# LDA모델에서 토픽 웨이트를 추출하여 데이터 프레임에 저장 
for num, topic in enumerate(topic_weight):
    for i in topic[0]:
        df_topic_weight.loc[num, 'topic' + str(i[0]+1)] = i[1]
        #df_topic_weight.loc[num, 'topic' + str(i[0])] = i[1] 

df_topic_weight = df_topic_weight.fillna(math.exp(-1000)) 

df_topic_last = pd.concat([DF_raw, df_dominant_topic, df_topic_weight], axis=1)
print(df_topic_last)

df_topic_last.to_excel(RESULT_TOPIC_EXCEL,sheet_name = "sheet1")

    id                                              title  \
0    1  윤석열 당선자가 주장한 지역별, 업종별 최저임금 차등적용의 근거가 되는 최저임금법 ...   
1    2                            여성가족부 폐지 찬성 및 동의에 관한 청원   
2    3                  중대범죄수사청 입법 조치 및 발족 시한의 법제화에 관한 청원   
3    4                                 여성가족부 폐지 반대에 관한 청원   
4    5              전력 판매 시장 민간 개방 반대를 위한 전기사업법 개정에 관한 청원   
5    6  윤석열 당선자의 대장동 부산저축은행 부실수사 봐주기 의혹과 김건희의 주가조작 실체의...   
6    7         포괄적 차별금지법을 우회 입법하려는 국가인권위원회법 개정안 반대에 관한 청원   
7    8                              인권정책기본법안 제정 반대에 관한 청원   
8    9                                    공직선거법 개정에 관한 청원   
9   10                             노점상 생계보호 특별법 제정에 관한 청원   
10  11  돌봄의 국가책임을 규정하고, 110만 돌봄노동자들의 처우개선을 담은 돌봄기본법 제정...   
11  12                                 농업·농촌·농민기본법에 관한 청원   
12  13             22년 3월 대선에서 수개표 시행을 위한 공직선거법 개정에 관한 청원   
13  14                          간호사 1인당 담당 환자 수 축소에 관한 청원   
14  15                               평등에 관한 법률안 반대에 관한 청원   
15  16           故손○○군 사

In [15]:
!zip -r /content/dataset3_result.zip /content/dataset

  adding: content/dataset/ (stored 0%)
  adding: content/dataset/1.Data_my/ (stored 0%)
  adding: content/dataset/1.Data_my/text.xlsx (deflated 6%)
  adding: content/dataset/1.Data_my/Data_stop_word.txt (deflated 35%)
  adding: content/dataset/1.Data_my/contents.xlsx (deflated 16%)
  adding: content/dataset/2.Model_my/ (stored 0%)
  adding: content/dataset/2.Model_my/Model_lda_topic_modeling.lda.id2word (deflated 48%)
  adding: content/dataset/2.Model_my/Model_lda_topic_modeling.lda.expElogbeta.npy (deflated 60%)
  adding: content/dataset/2.Model_my/Model_lda_topic_modeling.lda.state (deflated 23%)
  adding: content/dataset/2.Model_my/imsi.txt (stored 0%)
  adding: content/dataset/2.Model_my/Model_lda_topic_modeling.lda (deflated 71%)
  adding: content/dataset/3.Result_my/ (stored 0%)
  adding: content/dataset/3.Result_my/Result_corpus_excel.xlsx (deflated 12%)
  adding: content/dataset/3.Result_my/imsi.txt (stored 0%)
  adding: content/dataset/3.Result_my/Result_lda_vis.html (deflated

In [16]:
from google.colab import files
files.download('/content/dataset_result.zip')

FileNotFoundError: ignored